### Setup

In [1]:
import numpy as np

from hot_dust import preprocess, model

In [2]:
from importlib import reload

preprocess = reload(preprocess)
model = reload(model)

In [3]:
ds = preprocess.prepare_training_data()
train, validate, test = preprocess.to_tensorflow(preprocess.split_training_data(ds))

### Parameters

In [4]:
epochs = 500 # Max 500
batch_size = 64
buffer_size = 10 * batch_size

model = reload(model)  # DEBUG

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size)

layer = model.pretraining(dataset=test)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
network = model.compile(normalization=layer)
network.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 11)]              0         
                                                                 
 normalization (Normalizatio  (None, 11)               23        
 n)                                                              
                                                                 
 dense (Dense)               (None, 8)                 96        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 200
Trainable params: 177
Non-trainable params: 23
______________________________________________________________

### Fitting

In [6]:
import tensorflow as tf 
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", fit)

Epoch 1/500
3122/3122 - 3s - loss: 1.9270 - val_loss: 1.9025 - 3s/epoch - 1ms/step
Epoch 2/500
3122/3122 - 2s - loss: 1.9008 - val_loss: 1.8791 - 2s/epoch - 745us/step
Epoch 3/500
3122/3122 - 2s - loss: 1.8890 - val_loss: 1.8704 - 2s/epoch - 736us/step
Epoch 4/500
3122/3122 - 2s - loss: 1.8819 - val_loss: 1.8658 - 2s/epoch - 752us/step
Epoch 5/500
3122/3122 - 2s - loss: 1.8797 - val_loss: 1.8649 - 2s/epoch - 690us/step
Epoch 6/500
3122/3122 - 2s - loss: 1.8792 - val_loss: 1.8647 - 2s/epoch - 694us/step
Epoch 7/500
3122/3122 - 2s - loss: 1.8789 - val_loss: 1.8643 - 2s/epoch - 752us/step
Epoch 8/500
3122/3122 - 2s - loss: 1.8787 - val_loss: 1.8641 - 2s/epoch - 737us/step
Epoch 9/500
3122/3122 - 2s - loss: 1.8784 - val_loss: 1.8638 - 2s/epoch - 707us/step
Epoch 10/500
3122/3122 - 2s - loss: 1.8781 - val_loss: 1.8635 - 2s/epoch - 715us/step
Epoch 11/500
3122/3122 - 2s - loss: 1.8778 - val_loss: 1.8632 - 2s/epoch - 699us/step
Epoch 12/500
3122/3122 - 2s - loss: 1.8776 - val_loss: 1.8630 - 2

INFO:tensorflow:Assets written to: data/network\assets


INFO:tensorflow:Assets written to: data/network\assets


### Training and Validation Losses

In [7]:
model.plot_loss(fit)

:NdOverlay   [Variable]
   :Scatter   [epoch]   (value)